In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import cv2
import numpy as np
from ROIPoolingLayer import RoiPoolingConv
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import roi_helpers


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

img = cv2.imread('test.jpg')
h, w, _ = img.shape


img = cv2.resize(img, (224, 224))
img = np.expand_dims(img, 0)



Num GPUs Available:  2


In [2]:
def make_RPN(img_input, base_layers, anchors_num=9):        
    k = anchors_num
    x = layers.Conv2D(512, kernel_size=(3, 3),padding='same', activation='relu',
                      kernel_initializer='normal')(base_layers)
    reg = layers.Conv2D(4*k, kernel_size=(1, 1), activation='sigmoid')(x)
    scores = layers.Conv2D(1*k, kernel_size=(1, 1),activation='linear',
                           kernel_initializer='zero')(x)

    model = Model(inputs=img_input, outputs=[scores, reg])

    # model.summary()
    return model

In [3]:
# input_tensor = layers.Input(shape=(224,224,3))
img_input = layers.Input(shape=(None,None,3))
resnet = tf.keras.applications.ResNet50(input_tensor=img_input,include_top=False)
base_resnet_layer = resnet.get_layer('conv4_block6_out').output

print(resnet.summary())
# model_rpn = make_RPN(resnet.outputs[0])

# print(feature_map.shape)

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [4]:
print(resnet.outputs[0])
print(base_resnet_layer)

Tensor("conv5_block3_out/Identity:0", shape=(None, None, None, 2048), dtype=float32)
Tensor("conv4_block6_out/Identity:0", shape=(None, None, None, 1024), dtype=float32)


In [5]:
rpn= make_RPN(img_input,base_resnet_layer)

print(rpn.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [20]:
from tensorflow.keras.layers import Input, Add, Dense, Activation, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, \
    AveragePooling2D
from FixedBatchNormalization import FixedBatchNormalization 
def identity_block_td(input_tensor, kernel_size, filters, stage, block, trainable=True):

    # identity block time distributed
    nb_filter1, nb_filter2, nb_filter3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = TimeDistributed(Convolution2D(nb_filter1, (1, 1), trainable=trainable,
                                      kernel_initializer='normal'), name=conv_name_base + '2a')(input_tensor)
    x = TimeDistributed(FixedBatchNormalization(
        axis=bn_axis), name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = TimeDistributed(Convolution2D(nb_filter2, (kernel_size, kernel_size), trainable=trainable,
                                      kernel_initializer='normal', padding='same'), name=conv_name_base + '2b')(x)
    x = TimeDistributed(FixedBatchNormalization(
        axis=bn_axis), name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = TimeDistributed(Convolution2D(nb_filter3, (1, 1), trainable=trainable,
                                      kernel_initializer='normal'), name=conv_name_base + '2c')(x)
    x = TimeDistributed(FixedBatchNormalization(
        axis=bn_axis), name=bn_name_base + '2c')(x)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)

    return x


def conv_block_td(input_tensor, kernel_size, filters, stage, block, input_shape, strides=(2, 2), trainable=True):

    # conv block time distributed

    nb_filter1, nb_filter2, nb_filter3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = TimeDistributed(Convolution2D(nb_filter1, (1, 1), strides=strides, trainable=trainable,
                                      kernel_initializer='normal'), input_shape=input_shape, name=conv_name_base + '2a')(input_tensor)
    x = TimeDistributed(FixedBatchNormalization(
        axis=bn_axis), name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = TimeDistributed(Convolution2D(nb_filter2, (kernel_size, kernel_size), padding='same',
                                      trainable=trainable, kernel_initializer='normal'), name=conv_name_base + '2b')(x)
    x = TimeDistributed(FixedBatchNormalization(
        axis=bn_axis), name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = TimeDistributed(Convolution2D(nb_filter3, (1, 1), kernel_initializer='normal'),
                        name=conv_name_base + '2c', trainable=trainable)(x)
    x = TimeDistributed(FixedBatchNormalization(
        axis=bn_axis), name=bn_name_base + '2c')(x)

    shortcut = TimeDistributed(Convolution2D(nb_filter3, (1, 1), strides=strides, trainable=trainable,
                                             kernel_initializer='normal'), name=conv_name_base + '1')(input_tensor)
    shortcut = TimeDistributed(FixedBatchNormalization(
        axis=bn_axis), name=bn_name_base + '1')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x



def classifier_layers(x, input_shape, trainable=False):

    # compile times on theano tend to be very high, so we use smaller ROI pooling regions to workaround
    # (hence a smaller stride in the region that follows the ROI pool)
    x = conv_block_td(x, 3, [512, 512, 2048], stage=5, block='a',
                      input_shape=input_shape, strides=(2, 2), trainable=trainable)

    x = identity_block_td(x, 3, [512, 512, 2048],
                          stage=5, block='b', trainable=trainable)
    x = identity_block_td(x, 3, [512, 512, 2048],
                          stage=5, block='c', trainable=trainable)
    x = TimeDistributed(AveragePooling2D((7, 7)), name='avg_pool')(x)

    return x

In [21]:
from tensorflow.keras.layers import TimeDistributed, Dense, Flatten, Dropout, AveragePooling2D



def classifier(img_input, base_layers, input_rois, num_rois, nb_classes=21, trainable=False):    
    
    pooling_regions = 14
    input_shape = (num_rois, 14, 14, 1024)

    out_roi_pool = RoiPoolingConv(pooling_regions, num_rois)([
        base_layers, input_rois])
    out = classifier_layers(
        out_roi_pool, input_shape=input_shape, trainable=True)

    out = TimeDistributed(Flatten())(out)

    out_class = TimeDistributed(Dense(nb_classes, activation='softmax',
                                      kernel_initializer='zero'), name='dense_class_{}'.format(nb_classes))(out)
    # note: no regression target for bg class
    out_regr = TimeDistributed(Dense(4 * (nb_classes-1), activation='linear',
                                     kernel_initializer='zero'), name='dense_regress_{}'.format(nb_classes))(out)

    model = Model(inputs=[img_input, input_rois], outputs=[out_class, out_regr])

    return model
    

In [22]:
roi_input = layers.Input(shape=(None, 4))
class_model = classifier(img_input,base_resnet_layer, roi_input,32,trainable=True)

channels_last


In [23]:
model_all = Model(inputs= [img_input,roi_input],outputs=rpn.outputs + class_model.outputs)

In [24]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import losses

classes_count = 21
num_anchors = 9
optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)


rpn.compile(optimizer=optimizer, loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
class_model.compile(optimizer=optimizer_classifier, loss=[losses.class_loss_cls, losses.class_loss_regr(classes_count)], metrics={'dense_class_{}'.format(classes_count): 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

In [25]:
import data_augment
import data_generators
import config
import voc_parser

dataPath = 'dataset/voc2012/VOCdevkit'

C = config.Config()
all_imgs, classes_count, class_mapping = voc_parser.get_data(dataPath)


if 'bg' not in classes_count:
    classes_count['bg'] = 0
    class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping

inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
# pprint.pprint(classes_count)
print('Num classes (including bg) = {}'.format(len(classes_count)))


Processing 2011_001944.xml:   1%|          | 122/17125 [00:00<00:14, 1204.17it/s]

Parsing annotation files


Processing 2011_001315.xml:  17%|█▋        | 2974/17125 [00:02<00:10, 1406.86it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Processing 2011_000682.xml:  45%|████▌     | 7750/17125 [00:05<00:06, 1401.52it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Processing 2011_004735.xml:  76%|███████▌  | 12967/17125 [00:08<00:02, 1456.63it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in 

Training images per class:
Num classes (including bg) = 21


In [26]:
import random
import pprint

random.shuffle(all_imgs)

num_imgs = len(all_imgs)

train_imgs = [s for s in all_imgs if s['imageset'] == 'train']
val_imgs = [s for s in all_imgs if s['imageset'] == 'val']
test_imgs = [s for s in all_imgs if s['imageset'] == 'test']

print('Num train samples {}'.format(len(train_imgs)))
print('Num val samples {}'.format(len(val_imgs)))
print('Num test samples {}'.format(len(test_imgs)))


Num train samples 5717
Num val samples 5823
Num test samples 0


In [27]:
import tensorflow.keras.backend as K

def get_img_output_length(width, height):
    def get_output_length(input_length):
        # zero_pad
        input_length += 6
        # apply 4 strided convolutions
        filter_sizes = [7, 3, 1, 1]
        stride = 2
        for filter_size in filter_sizes:
            input_length = (input_length - filter_size + stride) // stride
        return input_length

    return get_output_length(width), get_output_length(height) 


data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, get_img_output_length, K.image_data_format(), mode='train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, get_img_output_length, K.image_data_format(), mode='val')
data_gen_test = data_generators.get_anchor_gt(test_imgs, classes_count, C, get_img_output_length, K.image_data_format(), mode='val')

In [28]:
print(data_gen_train)

<generator object get_anchor_gt at 0x7fd9e83c88d0>


In [29]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.python.keras.utils import generic_utils

import os

log_path = './logs'
if not os.path.isdir(log_path):
    os.mkdir(log_path)


callback = TensorBoard(log_path)
callback.set_model(model_all)

epoch_length = 1000
num_epochs = 2000
iter_num = 0
train_step = 0

losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []
# start_time = time.time()

best_loss = np.Inf

class_mapping_inv = {v: k for k, v in class_mapping.items()}
print('Starting training')

Starting training


In [30]:
X, Y, img_data = next(data_gen_train)
print(X.shape)
print(Y)
print(img_data)


(1, 802, 600, 3)
[array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ...,

In [31]:
def write_log(callback, names, logs, batch_no):
    for name, value in zip(names, logs):
        summary = tf.summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()


In [32]:

# vis = True
model_rpn = rpn

num_epochs = 500


for epoch_num in range(num_epochs):
    progbar = generic_utils.Progbar(epoch_length)   # keras progress bar 사용
    print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))

    while True:
        # try:
        # mean overlapping bboxes 출력
        if len(rpn_accuracy_rpn_monitor) == epoch_length and C.verbose:
            mean_overlapping_bboxes = float(sum(rpn_accuracy_rpn_monitor))/len(rpn_accuracy_rpn_monitor)
            rpn_accuracy_rpn_monitor = []
            print('Average number of overlapping bounding boxes from RPN = {} for {} previous iterations'.format(mean_overlapping_bboxes, epoch_length))
            if mean_overlapping_bboxes == 0:
                print('RPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')

        # data generator에서 X, Y, image 가져오기
        X, Y, img_data = next(data_gen_train)        
        
        
        loss_rpn = model_rpn.train_on_batch(X, Y)
        print(loss_rpn,train_step)
#         write_log(callback, ['rpn_cls_loss', 'rpn_reg_loss'], loss_rpn, train_step)

        P_rpn = model_rpn.predict_on_batch(X)

        R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_data_format(), use_regr=True, overlap_thresh=0.7, max_boxes=300)
        # note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
        X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, C, class_mapping)

        if X2 is None:
            rpn_accuracy_rpn_monitor.append(0)
            rpn_accuracy_for_epoch.append(0)
            continue

        # sampling positive/negative samples
        neg_samples = np.where(Y1[0, :, -1] == 1)
        pos_samples = np.where(Y1[0, :, -1] == 0)

        if len(neg_samples) > 0:
            neg_samples = neg_samples[0]
        else:
            neg_samples = []

        if len(pos_samples) > 0:
            pos_samples = pos_samples[0]
        else:
            pos_samples = []

        rpn_accuracy_rpn_monitor.append(len(pos_samples))
        rpn_accuracy_for_epoch.append((len(pos_samples)))

        if C.num_rois > 1:
            if len(pos_samples) < C.num_rois//2:
                selected_pos_samples = pos_samples.tolist()
            else:
                selected_pos_samples = np.random.choice(pos_samples, C.num_rois//2, replace=False).tolist()
            try:
                selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=False).tolist()
            except:
                selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=True).tolist()

            sel_samples = selected_pos_samples + selected_neg_samples
        else:
            # in the extreme case where num_rois = 1, we pick a random pos or neg sample
            selected_pos_samples = pos_samples.tolist()
            selected_neg_samples = neg_samples.tolist()
            if np.random.randint(0, 2):
                sel_samples = random.choice(neg_samples)
            else:
                sel_samples = random.choice(pos_samples)

        loss_class = class_model.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])
#         write_log(callback, ['detection_cls_loss', 'detection_reg_loss', 'detection_acc'], loss_class, train_step)
        train_step += 1

        losses[iter_num, 0] = loss_rpn[1]
        losses[iter_num, 1] = loss_rpn[2]

        losses[iter_num, 2] = loss_class[1]
        losses[iter_num, 3] = loss_class[2]
        losses[iter_num, 4] = loss_class[3]

        iter_num += 1

        progbar.update(iter_num, [('rpn_cls', np.mean(losses[:iter_num, 0])), ('rpn_regr', np.mean(losses[:iter_num, 1])),
                                  ('detector_cls', np.mean(losses[:iter_num, 2])), ('detector_regr', np.mean(losses[:iter_num, 3]))])

        if iter_num == epoch_length:
            loss_rpn_cls = np.mean(losses[:, 0])
            loss_rpn_regr = np.mean(losses[:, 1])
            loss_class_cls = np.mean(losses[:, 2])
            loss_class_regr = np.mean(losses[:, 3])
            class_acc = np.mean(losses[:, 4])

            mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
            rpn_accuracy_for_epoch = []

            if C.verbose:
                print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(mean_overlapping_bboxes))
                print('Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
                print('Loss RPN classifier: {}'.format(loss_rpn_cls))
                print('Loss RPN regression: {}'.format(loss_rpn_regr))
                print('Loss Detector classifier: {}'.format(loss_class_cls))
                print('Loss Detector regression: {}'.format(loss_class_regr))
                print('Elapsed time: {}'.format(time.time() - start_time))

            curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
            iter_num = 0
            start_time = time.time()

#             write_log(callback,
#                       ['Elapsed_time', 'mean_overlapping_bboxes', 'mean_rpn_cls_loss', 'mean_rpn_reg_loss',
#                        'mean_detection_cls_loss', 'mean_detection_reg_loss', 'mean_detection_acc', 'total_loss'],
#                       [time.time() - start_time, mean_overlapping_bboxes, loss_rpn_cls, loss_rpn_regr,
#                        loss_class_cls, loss_class_regr, class_acc, curr_loss],
#                       epoch_num)

            if curr_loss < best_loss:
                if C.verbose:
                    print('Total loss decreased from {} to {}, saving weights'.format(best_loss,curr_loss))
                best_loss = curr_loss
                model_all.save_weights(C.model_path)

            break

        # except Exception as e:
        #     print('Exception: {}'.format(e))
        #     continue    


Epoch 1/500
[7.043315, 6.963122, 0.08019321] 0


ValueError: in converted code:

    /home/jang/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_eager.py:305 train_on_batch  *
        outs, total_loss, output_losses, masks = (
    /home/jang/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_eager.py:253 _process_single_batch
        training=training))
    /home/jang/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_eager.py:167 _model_loss
        per_sample_losses = loss_fn.call(targets[i], outs[i])
    /home/jang/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/keras/losses.py:221 call
        return self.fn(y_true, y_pred, **self._fn_kwargs)
    /home/jang/study/tf/fastrcnn_keras/losses.py:56 class_loss_cls
        return lambda_cls_class * K.mean(categorical_crossentropy(y_true[0, :, :], y_pred[0, :, :]))
    /home/jang/anaconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/p

In [33]:
class_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
output = model_rpn.predict(X)

In [ ]:
print(output[0].shape)
print(output[1].shape)

In [ ]:
print(Y[0].shape)
print(Y[1].shape)

In [ ]:
print(model_rpn.outputs)

In [ ]:
print(model_rpn.outputs[0])
print(model_rpn.outputs[1])

In [ ]:
print(resnet.get_layer('conv4_block6_out').output)